<a href="https://colab.research.google.com/github/Hadrien-Cornier/cool-nn-stuff/blob/main/cleanlab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cleanlab[datalab] -q

In [16]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [19]:
import numpy as np
from transformers import pipeline
from cleanlab import Datalab

# Sample text data and labels with intentional label noise
texts = [
    "I love this product", "This is the worst service", "Amazing experience",
    "Not what I expected", "Terrible quality", "Excellent value for money",
    "I would buy this again", "Not worth the price", "Very satisfied",
    "Disappointing performance", "Highly recommend", "Would not recommend",
    "Exceeded my expectations", "Below average", "Fantastic design",
    "Poor customer service", "Great features", "Not as described",
    "Love the color", "Hate the material"
]
# Introduce some intentional label errors (e.g., flipping some labels)
labels = np.array([1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
labels[5] = 0  # Intentional error
labels[10] = 0 # Intentional error

# Use Hugging Face's sentiment analysis pipeline with a pretrained model
sentiment_analysis = pipeline("sentiment-analysis", model="maherh/google_electra_small_discriminator_sentiment_analysis")

# Get predicted probabilities for the entire dataset
pred_probs = []
for text in texts:
    result = sentiment_analysis(text)[0]
    # Convert label to probability format expected by Cleanlab
    if result['label'] == 'POSITIVE':
        pred_probs.append([result['score'], 1 - result['score']])
    else:
        pred_probs.append([1 - result['score'], result['score']])

pred_probs = np.array(pred_probs)

# Use Cleanlab to find label issues
lab = Datalab(data={'text': texts, 'labels':labels}, label_name='labels')
lab.find_issues(pred_probs=pred_probs)

# Report and get issues
lab.report(show_summary_score=True, show_all_issues=True)
label_issues = lab.get_issues()

# Display identified label issues
identified_label_issues = label_issues[label_issues["is_label_issue"] == True]
print(f"Cleanlab found {len(identified_label_issues)} potential label errors in the dataset.")

Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided pred_probs ...
Finding non_iid issues ...
Finding class_imbalance issues ...

Audit complete. 4 issues found in the dataset.
Dataset Information: num_examples: 20, num_classes: 2

Here is a summary of various issues found in your data:

     issue_type    score  num_issues
          label 0.750000           2
        outlier 0.013156           2
        non_iid 0.309500           0
class_imbalance 0.400000           0

(Note: A lower score indicates a more severe issue across all examples in the dataset.)

Learn about each issue: https://docs.cleanlab.ai/stable/cleanlab/datalab/guide/issue_type_description.html
See which examples in your dataset exhibit each issue via: `datalab.get_issues(<ISSUE_NAME>)`

Data indices corresponding to top examples of each issue are shown below.


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimat

In [20]:
identified_label_issues

,is_label_issue,label_score,is_outlier_issue,outlier_score,is_non_iid_issue,non_iid_score,is_class_imbalance_issue,class_imbalance_score
1,True,0.013230,False,0.006151,False,0.944379,False,1.0
10,True,0.014935,False,0.005901,False,0.867107,False,1.0
